In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df = pd.read_csv("dataset-loslagos-8months.csv", 
                 sep=";", 
                 engine='python')
df.columns = ["date", "media_outlet", "url", "title", "text"]
df["content"] = ""
df.content = df.title + " " + df.text
print("Largo del dataset: ", len(df))
df.head()

Largo del dataset:  29598


,date,media_outlet,url,title,text,content
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...,Reconocen a guardaparques de la Región de Los ...
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...,Con nuevos materiales comienza plan piloto en ...
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...,Centro de Salud Familiar CESFAM Puerto Varas i...
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...,Alcalde Tomás Gárate presidió por primera vez ...
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",Galería de Arte Machacoya realizará remate de ...


<img src="img/los_lagos.png" />

Fuente: [wikipedia](https://es.wikipedia.org/wiki/Portal:Regi%C3%B3n_de_Los_Lagos/Comunas_de_Los_Lagos)

In [3]:
comunas = ["ancud", "castro", "chonchi", "curaco de vélez", "dalcahue", 
           "puqueldón", "queilén", "quemchi", "quellón", "quinchao"
           "calbuco", "cochamó", "fresia", "frutillar", "llanquihue"
           "los muermos", "maullin", "puerto montt", "puerto varas", "osorno"
           "puerto octay", "purranque", "puyehue", "rio negro", "san juan de la costa",
           "san pablo", "chaitén", "futaleufú", "hualaihué", "palena"]

In [4]:
from tqdm.notebook import tqdm_notebook

df['comuna'] = ""
for index, row in tqdm_notebook(df.iterrows(), desc="buscando comunas en content"):
    found = [comuna for comuna in comunas if(comuna in str(row["content"]).lower())]
    if(len(found)!= 0): df.at[index, 'comuna'] = found


buscando comunas en content: 0it [00:00, ?it/s]

In [5]:
df['comuna'].value_counts()[1:10]

[puerto montt]                                 2418
[castro]                                       1148
[puerto varas]                                  708
[ancud]                                         543
[puerto montt, puerto varas]                    270
[castro, puerto montt]                          205
[quellón]                                       205
[purranque]                                     191
[ancud, castro, chonchi, dalcahue, quellón]     183
Name: comuna, dtype: int64

In [6]:
selection = ['ancud']
mask = df.comuna.apply(lambda x: any(item for item in selection if item in x))

df1 = df[mask]
df1.head()

,date,media_outlet,url,title,text,content,comuna
7,2021-10-01,seminariolocal,https://www.semanariolocal.cl/archivos/2207,Gremio médico rechaza cierre de camas críticas...,El Capítulo Médico Ancud y el Consejo Regional...,Gremio médico rechaza cierre de camas críticas...,"[ancud, puerto montt]"
33,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Acercan atención de salud a los vecinos de Tri...,El corte de cinta de la Estación Médico Rural ...,Acercan atención de salud a los vecinos de Tri...,"[ancud, quellón]"
34,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Puqueldón avanza a apertura en el fin del Esta...,"El avance de Puqueldón a la apertura inicial, ...",Puqueldón avanza a apertura en el fin del Esta...,"[ancud, castro, puqueldón, quellón]"
36,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Celebran Día de la Rabia con acciones preventivas,Una demostración de adiestramiento de mascotas...,Celebran Día de la Rabia con acciones preventi...,"[ancud, quemchi]"
39,2021-10-01,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,denuncia y ayuda,no espere de más por atenciones en cesfam.- El...,denuncia y ayuda no espere de más por atencion...,"[ancud, castro, chonchi, dalcahue, quellón]"


### LDA

In [ ]:
import spacy
nlp = spacy.load("es_core_news_md")

def spacy_tokenizer(texto):
    # crea lista de tokens a partir de un texto
    mytokens = []
    doc = nlp(texto.lower().strip())
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space and not token.is_digit and not token.like_num:
            mytokens.append(token.lemma_)
    return mytokens

In [ ]:
#Utilizando Kmeans y tf-idf para realizar un primer agrupamiento de las noticias
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

myStopWords=['gnl','=','$','+','a','e','o','u','y',' ','  ','   ','    ']
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer, stop_words=myStopWords, ngram_range=(1,1))

X=vectorizer.fit_transform(df1.content) 
X_term = vectorizer.get_feature_names()



In [ ]:
import numpy as np
print(df1.content[:1])

print(len(X_term))
print(X_term[:20])
print(X.shape)
print(X[:1])

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


num_k = range(1, 10)
inertias = []

for k in num_k:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k, n_init=10)

    # Fit model to samples
    model.fit(X)

    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)


# Plot ks vs inertias
plt.plot(num_k, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(num_k)
plt.show()

In [ ]:
from matplotlib.pylab import hist, show
true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    cluster_words = np.array(terms)[order_centroids[i, :15]]
    print('Cluster {}: {}'.format(i, '-'.join(cluster_words)))


In [ ]:
#Aplicando Modelos Probabilistas de Tópicos y LDA
from sklearn.decomposition import LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
    

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                stop_words=myStopWords,tokenizer=spacy_tokenizer, ngram_range=(1,1))
tf = tf_vectorizer.fit_transform(df1.content)
diccionario= tf_vectorizer.get_feature_names()


perp = []
for k in num_k:
    #Estimación de LDA con Bayes Variacional
    lda = LatentDirichletAllocation(n_components=k, max_iter=10,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

    lda.fit(tf)
    #Cálculo de índice de ajuste de los datos
    perp.append(lda.perplexity(tf))

plt.plot(num_k, perp, '-o')
plt.xlabel('number of topics, k')
plt.ylabel('perplexity')
plt.xticks(num_k)
plt.show()

#print("\nTopics in LDA model:")
#print_top_words(lda, diccionario, 15)

In [ ]:
#Generando algunos gráficos de los resultados
from matplotlib.pylab import hist, show
import seaborn as sns

lda = LatentDirichletAllocation(n_components=true_k, max_iter=1000,
                                learning_method='online',
                                learning_offset=10.,
                                random_state=0)
y=lda.fit_transform(tf)

print(y)
#distribución de tópicos en todos los documentos
dist = sum(y)/len(noticias)
print(dist)

w=np.arange(0,true_k)
print(w)
plt.bar(w,height=dist,alpha=0.5,tick_label=w)
show()
#distribución de tópicos en el documento 51
plt.bar(w,height=y[50],alpha=0.5,tick_label=w)
show()


#frecuencia de palabras por tópico
n_top_words=15
numo =np.arange(0,n_top_words)
for idx,topic in enumerate(lda.components_):
    d =[]
    s = []
    for i in topic.argsort()[:-n_top_words - 1:-1]:
        s.append(diccionario[i])
        d.append(topic[i])
    
    plt.barh(numo,width=d,alpha=0.5,tick_label=s)
    show()


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
prepared = pyLDAvis.sklearn.prepare(lda,tf,tf_vectorizer)
pyLDAvis.display(prepared)

In [ ]:
import lda

model = lda.LDA(n_topics=true_k, n_iter=5000, alpha=0.1, eta=0.1, random_state=3)
model.fit(tf)  # model.fit_transform(X) is also available

topic_word = model.topic_word_  # model.components_ also works
n_top_words=10
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(diccionario)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    

In [ ]:
plt.plot(model.loglikelihoods_[5:])

In [ ]:
#Generando algunos gráficos de los resultados
from matplotlib.pylab import hist, show

y=model.doc_topic_
#distribución de tópicos en todos los documentos
dist = sum(y)/len(noticias)
w=np.arange(0,true_k)
plt.bar(w,height=dist,alpha=0.5,tick_label=w)
show()

#distribución de tópicos en el documento 100
plt.bar(w,height=y[51],alpha=0.5,tick_label=w)
show()

#frecuencia de palabras por tópico
n_top_words=15
numo =np.arange(0,n_top_words)

for idx,topic in enumerate(model.topic_word_):
    d =[]
    s = []
    for i in topic.argsort()[:-n_top_words - 1:-1]:
        s.append(diccionario[i])
        d.append(topic[i])
    
    plt.barh(numo,width=d,alpha=0.5,tick_label=s)
    show()

In [ ]:
#calculando cluster en base al tópico mas frecuente
z=[]
for i in range(len(noticias)):
    z.append(np.argmax(y[i]))
plt.scatter(range(88),z,c=z)
show()

#distribución de cluster de acuerdo a la definición anterior
hist(z,bins=np.arange(-0.5,5.6,1),alpha=0.5,rwidth=0.9,density=True)
show()

In [ ]:
prepared = pyLDAvis.sklearn.prepare(model,tf,tf_vectorizer)
pyLDAvis.display(prepared)

### [Topic Modelling in Embedding Spaces](https://github.com/lffloyd/embedded-topic-model)

In [ ]:
import torch 
print(torch.__version__)

In [ ]:
#!pip install -U embedded_topic_model

In [ ]:
from embedded_topic_model.utils import preprocessing

# Preprocessing the dataset
vocabulary, train_dataset, _, = preprocessing.create_etm_datasets(
    ancud.content, 
    min_df=0.01, 
    max_df=0.75, 
    train_size=0.85, 
)

In [ ]:
from embedded_topic_model.utils import embedding

# Training word2vec embeddings
embeddings_mapping = embedding.create_word2vec_embedding_from_dataset(ancud.content)

In [ ]:
from embedded_topic_model.models.etm import ETM

# Training an ETM instance
etm_instance = ETM(
    vocabulary,
    embeddings=embeddings_mapping, # You can pass here the path to a word2vec file or
                                   # a KeyedVectors instance
    num_topics=8,
    epochs=300,
    debug_mode=True,
    train_embeddings=False, # Optional. If True, ETM will learn word embeddings jointly with
                            # topic embeddings. By default, is False. If 'embeddings' argument
                            # is being passed, this argument must not be True
)

etm_instance.fit(train_dataset)

In [ ]:
topics = etm_instance.get_topics(20)
topic_coherence = etm_instance.get_topic_coherence()
topic_diversity = etm_instance.get_topic_diversity()